In [1]:
import pandas as pd
import json
from SPARQLWrapper import SPARQLWrapper, JSON



In [2]:
def runSparql(query):
    sparql = SPARQLWrapper('https://query.wikidata.org/sparql')
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [ ]:
# All movies Bud Spencer plays in
runSparql("""
    SELECT ?item ?itemLabel (MIN(?date) AS ?firstReleased)
    WHERE {
      ?item wdt:P161 wd:Q221074;
            wdt:P577 ?date
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    } GROUP BY ?item ?itemLabel
    ORDER BY (?date)
    """)

In [3]:
# All actors of movies Bud Spencer participated in
allActors = runSparql("""
    SELECT ?actor ?actorLabel ?movie ?movieLabel (MAX(?date) AS ?firstReleased)
    WHERE {
      ?movie wdt:P161 wd:Q221074 .
      ?movie wdt:P161 ?actor;
      wdt:P577 ?date
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    } GROUP BY ?actor ?actorLabel ?movie ?movieLabel
    ORDER BY ?date
""")
allActors

,actor,actorLabel,movie,movieLabel,firstReleased
0,http://www.wikidata.org/entity/Q221074,Bud Spencer,http://www.wikidata.org/entity/Q116187,Thieves and Robbers,1983-02-25T00:00:00Z
1,http://www.wikidata.org/entity/Q432638,Tomás Milián,http://www.wikidata.org/entity/Q116187,Thieves and Robbers,1983-02-25T00:00:00Z
2,http://www.wikidata.org/entity/Q928456,Marc Lawrence,http://www.wikidata.org/entity/Q116187,Thieves and Robbers,1983-02-25T00:00:00Z
3,http://www.wikidata.org/entity/Q993165,Bruno Corbucci,http://www.wikidata.org/entity/Q116187,Thieves and Robbers,1983-02-25T00:00:00Z
4,http://www.wikidata.org/entity/Q3783419,Harold Bergman,http://www.wikidata.org/entity/Q116187,Thieves and Robbers,1983-02-25T00:00:00Z
...,...,...,...,...,...
831,http://www.wikidata.org/entity/Q465089,Giulio Base,http://www.wikidata.org/entity/Q32999881,They called him Bulldozer,2017-07-27T00:00:00Z
832,http://www.wikidata.org/entity/Q505793,Alessandro Capone,http://www.wikidata.org/entity/Q32999881,They called him Bulldozer,2017-07-27T00:00:00Z
833,http://www.wikidata.org/entity/Q538488,Guido De Angelis,http://www.wikidata.org/entity/Q32999881,They called him Bulldozer,2017-07-27T00:00:00Z
834,http://www.wikidata.org/entity/Q930042,Maurizio De Angelis,http://www.wikidata.org/entity/Q32999881,They called him Bulldozer,2017-07-27T00:00:00Z


In [57]:
allActors.query('actorLabel == "Bud Spencer"').actorLabel.count()

59

In [4]:
allActors.hist(column='actorLabel')

ValueError: hist method requires numerical columns, nothing to plot.

In [16]:
import altair as alt

for i, row in allActors.iterrows():
    actor = row['actorLabel']
    allActors.at[i, 'amount'] = allActors.query('actorLabel == "{}"'.format(actor)).actorLabel.count()


In [15]:


alt.Chart(allActors).mark_tick().encode(
    x='firstReleased:T',
    y=alt.Y('actorLabel:O')
)

alt.Chart(...)

In [8]:
runSparql("""
        SELECT *
        WHERE {
          ?item wdt:P161 wd:Q221074;
                wdt:P577 ?date
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        } GROUP BY ?item ?itemLabel
        ORDER BY (?date)
    """)

QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'SPARQL-QUERY: queryStr=\n        SELECT *\n        WHERE {\n          ?item wdt:P161 wd:Q221074;\n                wdt:P577 ?date\n          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }\n        } GROUP BY ?item ?itemLabel\n        ORDER BY (?date)\n    \njava.util.concurrent.ExecutionException: org.openrdf.query.MalformedQueryException: Wildcard not allowed with aggregate.\n\tat java.util.concurrent.FutureTask.report(FutureTask.java:122)\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:206)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServlet.doGet(RESTServlet.java:240)\n\t

In [36]:
import altair as alt

alt.Chart(getBudSpencerMovies()).mark_line().encode(
    x='firstReleased',
    y='duration'
)

alt.Chart(...)